# Genetic Algorithm 
## Investigación de Operaciones, ITAM (2023). 

@author: Diego Velázquez Trejo

@author: Viviana de los Ríos


### Required libraries

In [1]:
# Library for matrix operations
import numpy as np 
# Library for time operations
import time 
# Library for random operations
import random 
# Library for plotting
from matplotlib import pyplot as plt 

# Auxiliar functions 

In [2]:
# Function that prints the population 
def print_pop(population): 
  print("Population: ")
  for i in range(len(population)): 
    if(i % 10 == 0): 
      print("\n")
    print(population[i].toString(), end = ", ")


# Reading the data for the Genetic Algorithm 

In [3]:

'''
Programa Main para la optimización del problema de la mochila
@author Diego Velázquez Trejo

'''
from leyendo_datos_mochila import CONJUNTO_OBJETOS
from leyendo_datos_mochila import CAPACIDAD
from leyendo_datos_mochila import OBJETOS_TOTALES # Va a ser importante tener esta variable en mente cuando intentemos crear tantas mochilas con objetos y que sean más objetos de los que predisponemos 
from Mochila import Mochila
import random

conjunto_identificadores = set() # Contiene los identificadores de los objetos que se pueden insertar en las mochilas (será una variable auxiliar)

for i in range(1, len(CONJUNTO_OBJETOS)+1):
    conjunto_identificadores.add(str(i))


# Necesito una función para generar una mochila inicial VÁLIDA
def genera_mochila_valida(informacion = CONJUNTO_OBJETOS, capacidad = CAPACIDAD):
    # Copia del conjunto
    M0 = Mochila(set(), capacidad)
    el = conjunto_identificadores.pop()
    while(M0.agrega_elemento(el) and len(conjunto_identificadores) > 0):
        el = conjunto_identificadores.pop()
    # Agregamos el último elemento que se sacó del ciclo while y que ya no entró a la mochila 
    conjunto_identificadores.add(el)
    return M0


# Vemos los resultados de una mochila inicial y del conjunto de identificadores
print(f"\nTamaño del conjunto de potenciales objetos: {len(conjunto_identificadores)}\n\n")
# Vamos a generar una población de 100 mochilas 
initialPop = []

total_elementos_agregados_a_mochila = 0 

size_pop = 50

for i in range(size_pop):
    M = genera_mochila_valida()
    initialPop.append(M)
    total_elementos_agregados_a_mochila += M.longitud()

# Vamos a imprimir cada mochila 

print_pop(initialPop)
print(f"\n\n\nCantidad de objetos que están disponibles para configurar más mochilas: {len(conjunto_identificadores)}")
print(f"Cantidad de objetos dentro de las mochilas: {total_elementos_agregados_a_mochila}\n")



Tamaño del conjunto de potenciales objetos: 10000


Population: 


Capacidad: 1000000, #Elemetos: 9, Peso: 938153, Valor:913884 : {'8123', '6252', '4636', '8886', '9091', '6810', '227', '1670', '2460'}, Capacidad: 1000000, #Elemetos: 10, Peso: 962376, Valor:949461 : {'1414', '1548', '5742', '7808', '654', '745', '4098', '2910', '9061', '2407'}, Capacidad: 1000000, #Elemetos: 8, Peso: 842952, Valor:854427 : {'761', '202', '8673', '7880', '2905', '4074', '2255', '3863'}, Capacidad: 1000000, #Elemetos: 11, Peso: 858901, Valor:859831 : {'7866', '3725', '619', '9402', '5529', '9136', '3809', '6749', '9654', '801', '3205'}, Capacidad: 1000000, #Elemetos: 9, Peso: 858425, Valor:856479 : {'8712', '8677', '5077', '6290', '8701', '5899', '9895', '5303', '7085'}, Capacidad: 1000000, #Elemetos: 9, Peso: 988881, Valor:976239 : {'7030', '1362', '7776', '5981', '1461', '1242', '7626', '7115', '3588'}, Capacidad: 1000000, #Elemetos: 10, Peso: 993692, Valor:972597 : {'1949', '3804', '8402', '7567', '6

# Genetic functions

In [4]:
# Función para agregar un elemento a la mochila 
def agrega_elemento_mochila(mochila, iters): 
    # En caso de haber llegado al máximo de llamadas recursivas intentando agregar un elemento, regresamos False
    if(iters > 3): 
        return False
    # Extraemos un elemento aleatorio del conjunto de identificadores
    elemento = random.choices(list(conjunto_identificadores),k=1)[0]
    if(mochila.agrega_elemento(elemento)):
        # Eliminamos elemento del conjunto de identificadores
        conjunto_identificadores.remove(elemento)
        return True 
    else: 
        return agrega_elemento_mochila(mochila, iters + 1)

# Recordar que el objeto mochila posee los siguientes métodos: obtiene_elemento y agrega_elemento
# En donde obtiene_elemento extraer un elemento aleatorio de la mochila (eliminándolo de la misma). 
# También tiene: longitud (regresa la cantidad de elementos que posee)
# Tambiém tiene: lista_elementos (para obtener los identificadores de los objetos que están dentro de la mochila)

def CrossOver(mochila1, mochila2, crossRate):
    pass 

def CrossOverToss(mochila1, mochila2, crossRate):
    pass

# Vamos a utilizar únicamente este método de cruce ya que aquí no importa la posición de los objetos que conforman a la mochila 
def CrossOverProbabilistic(mochila1, mochila2, crossRate):
    # Caso en donde no se lleva a cabo la mutación 
    if(random.random() > crossRate):
        return mochila1, mochila2
        
    # Vamos a obtener los elementos de la mochila1 y los elementos de la mochila2
    elementos_mochila1 = mochila1.lista_elementos()
    elementos_mochila2 = mochila2.lista_elementos()

    # Vamos a concatenar la lista de elementos_mochila1 y elementos_mochila2
    elementos = elementos_mochila1 + elementos_mochila2

    # Vamos a generar dos nuevas mochilas
    nueva_mochila1 = Mochila(set(), CAPACIDAD)
    nueva_mochila2 = Mochila(set(), CAPACIDAD)

    for elemento in elementos: 
        if(random.random() < 0.5): 
            res = nueva_mochila1.agrega_elemento(elemento)
            if(res == False and nueva_mochila2.agrega_elemento(elemento) == False): 
                    conjunto_identificadores.add(elemento)
        else: 
            res = nueva_mochila2.agrega_elemento(elemento)
            # Intentamos ahora colocarlo en la primera mochila 
            if(res == False and nueva_mochila1.agrega_elemento(elemento) == False):
                conjunto_identificadores.add(elemento)
    return nueva_mochila1, nueva_mochila2

# Función que escogerá un índice aleatorio del conjunto de identificadores y lo agregará a la mochila, quitando uno 
def MutateElement(mochila, mutationRate):
    if(random.random() < mutationRate):
        # Removemos un elemento aleatorio de la mochila 
        elemento = mochila.obtiene_elemento()
        # Tenemos que agregar un nuevo elemento del conjunto_identificadores (siempre y cuando siga respetando la capacidad de la mochila)
        res = agrega_elemento_mochila(mochila, 0)
        # En caso que sí se haya podido agregar un elemento, entonces agregamos el elemento que se sacó de la mochila al conjunto de identificadores
        if(res):
            conjunto_identificadores.add(elemento)
        else: 
            # En caso que no se haya podido agregar un nuevo elemento a la mochila, entonces agregamos el elemento que se sacó de la mochila a la mochila
            mochila.agrega_elemento(elemento)

    return mochila        

# Función que se encargará de iterar sobre los identificadores de la mochila y cambiar uno, dependiendo un u ~ U(0,1)
def MutateElementInversion(mochila, mutationRate):
    pass 

# Vamos a definir los métodos por fuera y pasarlos como parámetros
MetodosCruza = {
    "cross.Over": CrossOver, 
    "cross.Over.Toss": CrossOverToss,
    "cross.over.probabilistic": CrossOverProbabilistic
}

MetodosMutacion = {
    "uniform.mutation": MutateElement,
    "inversion.mutation": MutateElementInversion
}

# Pruebas de los dos operadores que tenemos

In [5]:

# Pruebas unitarias para el operador de cruza 
for i in range(40): 
    # Vamos a hacer unas pruebas con algunas mochilas de la initialPop 
    mochila1_index = random.randint(0, len(initialPop) - 1)
    mochila2_index = random.randint(0, len(initialPop) - 1)

    mochila1 = initialPop[mochila1_index]
    mochila2 = initialPop[mochila2_index]

    # Vamos a obtener la suma de los objetos en la población 
    suma_elementos_en_poblacion = 0 
    for mochila in initialPop:
        suma_elementos_en_poblacion += mochila.longitud()
    len_conjunto_ids_antes_cruza = len(conjunto_identificadores)

    # Vamos a hacer una prueba con el método de cruce probabilístico
    mochila1, mochila2 = CrossOverProbabilistic(mochila1, mochila2, 0.3)
    print(mochila1.toString(), mochila2.toString())
    initialPop[mochila1_index] = mochila1
    initialPop[mochila2_index] = mochila2

    # Vamos a ver que la suma de todos los objetos de las mochilas de la población más la suma de la longitud del conjunto de identificadores sea igual a 10000 
    suma_despues_cruza = 0
    for mochila in initialPop:
        suma_despues_cruza += mochila.longitud()

    len_conjunto_ids_desp_cruza = len(conjunto_identificadores)

    if(not (suma_despues_cruza + len_conjunto_ids_desp_cruza == 10000 and suma_elementos_en_poblacion + len_conjunto_ids_antes_cruza == 10000)):
        print("Algo salió mal")
        print(f"Antes de la mutación: {suma_elementos_en_poblacion}, {len_conjunto_ids_antes_cruza}")
        print(f"Después de la mutación: {suma_despues_cruza}, {len_conjunto_ids_desp_cruza}")


Capacidad: 1000000, #Elemetos: 10, Peso: 853432, Valor:818586 : {'3249', '7537', '7261', '2141', '9179', '8480', '1396', '8095', '463', '7801'} Capacidad: 1000000, #Elemetos: 10, Peso: 955492, Valor:966436 : {'3167', '9306', '9461', '9268', '2289', '7463', '6425', '4558', '8984', '973'}
Capacidad: 1000000, #Elemetos: 6, Peso: 882450, Valor:868533 : {'5527', '3366', '7229', '1446', '3334', '3621'} Capacidad: 1000000, #Elemetos: 11, Peso: 937533, Valor:949795 : {'9107', '2221', '3860', '6641', '8725', '9437', '6945', '3223', '6566', '6962', '7388'}
Capacidad: 1000000, #Elemetos: 12, Peso: 891941, Valor:870026 : {'3868', '6470', '7173', '6460', '385', '5180', '4920', '8623', '251', '1437', '1818', '9421'} Capacidad: 1000000, #Elemetos: 8, Peso: 842952, Valor:854427 : {'761', '202', '8673', '7880', '2905', '4074', '2255', '3863'}
Capacidad: 1000000, #Elemetos: 10, Peso: 906986, Valor:898174 : {'4723', '4511', '7897', '7156', '6281', '214', '740', '8708', '7357', '4952'} Capacidad: 1000000,

In [6]:
# Pruebas unitarias para el operador de mutación 
for i in range(40):    

    mochila1_index = random.randint(0, len(initialPop) - 1)
    mochila1 = initialPop[mochila1_index]

    # Vamos a obtener la suma de los objetos en la población 
    suma_elementos_en_poblacion = 0 
    for mochila in initialPop:
        suma_elementos_en_poblacion += mochila.longitud()
    len_conjunto_ids_antes_cruza = len(conjunto_identificadores)

    # Vamos a hacer una prueba con el método de mutación de elemento
    mochila1 = MutateElement(mochila1, 1)
    print(mochila1.toString())

    initialPop[mochila1_index] = mochila1

    # Vamos a ver que la suma de todos los objetos de las mochilas de la población más la suma de la longitud del conjunto de identificadores sea igual a 10000 
    suma_despues_mutacion = 0
    for mochila in initialPop:
        suma_despues_mutacion += mochila.longitud()

    len_conjunto_ids_desp_cruza = len(conjunto_identificadores)

    if(not (suma_despues_mutacion + len_conjunto_ids_desp_cruza == 10000 and suma_elementos_en_poblacion + len_conjunto_ids_antes_cruza == 10000)):
        print("Algo salió mal")
        print(f"Antes de la mutación: {suma_elementos_en_poblacion}, {len_conjunto_ids_antes_cruza}")
        print(f"Después de la mutación: {suma_despues_mutacion}, {len_conjunto_ids_desp_cruza}")




Capacidad: 1000000, #Elemetos: 8, Peso: 866890, Valor:839091 : {'5956', '5899', '217', '9352', '4523', '5303', '4762', '4590'}
Capacidad: 1000000, #Elemetos: 11, Peso: 902368, Valor:889877 : {'3249', '490', '1588', '2365', '7537', '4424', '9179', '8480', '1960', '8095', '7801'}
Capacidad: 1000000, #Elemetos: 10, Peso: 919126, Valor:896754 : {'8750', '8518', '7326', '6950', '3076', '3298', '7333', '823', '9562', '6339'}
Capacidad: 1000000, #Elemetos: 9, Peso: 878580, Valor:850934 : {'8123', '6252', '2965', '8774', '7524', '8037', '6700', '8537', '1670'}
Capacidad: 1000000, #Elemetos: 11, Peso: 967654, Valor:941391 : {'2974', '2490', '4636', '1022', '8886', '2460', '6810', '3479', '1424', '8933', '9091'}
Capacidad: 1000000, #Elemetos: 10, Peso: 849442, Valor:818267 : {'8837', '8816', '7424', '3757', '3553', '1057', '9405', '699', '9829', '3933'}
Capacidad: 1000000, #Elemetos: 6, Peso: 916629, Valor:889713 : {'6415', '6865', '1978', '8353', '4906', '3145'}
Capacidad: 1000000, #Elemetos: 8